In the name of god

Ali Ataollahi / 810199461

AI_FALL_1401 / CA1


## Table of Contents
* [Import Dependencies](##Import-Dependencies)
* [Const Vars](##Const-Vars)
* [Add Data Classes](##Add-Data-Classes)
* [Add Utils](##Add-Utils)
* [Read From File](##Read-From-File)
* [BFS](##BFS)
    * [Modeling BFS](##Modeling-BFS)
    * [Test BFS](##Test-BFS)
        * [Test BFS Results](##Test-BFS-Results)
* [IDS](##IDS)
    * [Modeling IDS](##Modeling-IDS)
    * [Test IDS](##Test-IDS)
        * [Test IDS Results](##Test-IDS-Results)
* [A*](##A*)
    * [Modeling A*](##Modeling-A*)
    * [Test A*](##Test-A*)
        * [Test A* Results](##Test-A*-Results)
* [Weighted A*](##Weighted-A*)
    * [Modeling Weighted A*](##Modeling-Weighted-A*)
    * [Weighted A* 1](##Weighted-A*-1)
        * [Weighted A* 1 Results](##Weighted-A*-1-Results)
    * [Weighted A* 2](##Weighted-A*-2)
        * [Weighted A* 2 Results](##Weighted-A*-2-Results)


## Import Dependencies

In [1]:
from dataclasses import dataclass
from collections import deque
import time
import heapq

## Const Vars

In [2]:
normal_type = 0
hard_type = 1
recipe_type = 2
morid_type = 3

class bcolors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    RED = '\033[91m'
    WHITE = '\033[0m'


## Add Data Classes

In [3]:

@dataclass
class State:
    position : int
    spent_time : int 
    wait_time : int
    path : list[int]
    hard_nodes : dict[int, int]
    done_recipes : set[int]
    done_morids : set[int]
    
    def __eq__(self, other):
        if isinstance(other, State):
            if other.position == self.position and \
               other.done_recipes == self.done_recipes and \
               other.done_morids == self.done_morids:
                return True
        return False

## Add Utils

In [4]:
class Utils():
    def __init__(self):
        pass

    def parseLine(self, line, delimiter=' '):
        index = line.index(delimiter) if delimiter in line else None
        if index is None:
            return [line, None]
        result = line[0:index]
        remainingLine = line[index + 1:]
        return [result, remainingLine]
    
    def print_state(self, state, file_name, state_counter):
        print("Path :",' -> '.join(map(str, state.path)))
        print("States Count :",state_counter)
        print("Spent Time :",state.spent_time)

        
    def timer_func(self, func):
        def wrap_func(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            print(f'executed in {(end-start)} sec')
            print(f"testing {bcolors.RED}ends{bcolors.WHITE}")
            print()
            return result
        return wrap_func

## Read From File

In [5]:
utils = Utils()

def read_file(file_name):
    global neighs, hard_nodes_list, recipe_nodes_list, morid_nodes_list, seyed_position, morid_nodes_num, recipe_nodes_num, morids_recipes, hard_nodes
    neighs, hard_nodes_list, recipe_nodes_list, morid_nodes_list, seyed_position, morid_nodes_num, recipe_nodes_num = [], [], [], [], 0, 0, 0
    morids_recipes, hard_nodes = dict(), dict()
    with open(file_name, 'r') as test_file:
        lines = test_file.read()
        line, lines = utils.parseLine(lines, '\n')
        vertices , edges = map(int,line.split(' '))
        
        neighs = [[] for i in range(vertices)]
        for i in range(edges):
            line, lines = utils.parseLine(lines, '\n')
            position1, position2 = map(int,line.split(' '))
            neighs[position1 - 1].append(position2)
            neighs[position2 - 1].append(position1)
            
        nodes_types = [normal_type for i in range(vertices)] 
        hard_nodes_list = nodes_types.copy()
        recipe_nodes_list = nodes_types.copy()
        morid_nodes_list = nodes_types.copy()

        line, lines = utils.parseLine(lines, '\n')
        hard_nodes_num = int(line)
        line, lines = utils.parseLine(lines, '\n')
        for i in range(hard_nodes_num):
            arg, line = utils.parseLine(line, ' ')
            hard_node_position = int(arg)
            hard_nodes_list[hard_node_position - 1] = hard_type
            hard_nodes[hard_node_position] = 0
        
        line, lines = utils.parseLine(lines, '\n')
        morid_nodes_num = int(line)
        for i in range(morid_nodes_num):
            line, lines = utils.parseLine(lines, '\n')
            arg, line = utils.parseLine(line, ' ')
            morid_position = int(arg)
            morids_recipes[morid_position] = set()
            morid_nodes_list[morid_position - 1] = morid_type

            arg, line = utils.parseLine(line, ' ')
            recipes_num = int(arg)
            for j in range(recipes_num):
                arg, line = utils.parseLine(line, ' ')
                recipe_position = int(arg)
                recipe_nodes_list[recipe_position - 1] = recipe_type
                morids_recipes[morid_position].add(recipe_position)
                
                

        line, lines = utils.parseLine(lines, '\n')
        seyed_position = int(line)

    recipe_nodes_num = 0
    for node in recipe_nodes_list:
        if node == recipe_type:
            recipe_nodes_num += 1

## BFS

## Modeling BFS
We use state for path finding and state hase many fields (shown in its dataclass)
Also we use frontier_states (queue) and explored_states (list)

- Initial State : State(position : init_position ,
                        spent_time : 0 ,
                        wait_time : 0 ,
                        path : empty_list ,
                        hard_nodes : empty_list_dict ,
                        done_recipes : empty_set ,
                        done_morids : empty_set)
                        
- Actions : we can append neighs of last node of frontier_states to first of that
- Transition Model : we expand nodes that are not expanded and not in frontier_states to achive goal state
- Goal state : in goal state we have done_morids is morid_nodes_num and wait_time is 0
- Path Cost : spent_time must be path cost

In [6]:
def is_goal(state):
    return True if len(state.done_morids) == morid_nodes_num and state.wait_time == 0 else False

@utils.timer_func
def bfs(file_name):
    state_counter = 1
    file_name_ = file_name.replace('.txt', '')
    print(f"testing {bcolors.BLUE}bfs{bcolors.WHITE} for {bcolors.GREEN}{file_name_}{bcolors.WHITE} started ...")
    read_file(file_name)
    frontier_states = deque()
    explored_states = []
    first_state = State(seyed_position, 0, 0, [seyed_position], hard_nodes, [], [])
    frontier_states.appendleft(first_state)
    while(True):
        state = frontier_states.pop()
        if(is_goal(state)):
            utils.print_state(state, file_name, state_counter)
            return
        ### decrease wait time
        if state.wait_time != 0:
            state.wait_time -= 1
            state.spent_time += 1
            frontier_states.appendleft(state)
            continue
        
        explored_states.append(state)
        neighs_position = neighs[state.position - 1]
        for neigh_position in neighs_position:
            
            new_path = state.path.copy()
            new_morids_list = state.done_morids
            new_recipes_list = state.done_recipes
            new_hard_nodes_list = state.hard_nodes.copy()
            wait_time = 0
            
            new_path.append(neigh_position)
            
            if recipe_nodes_list[neigh_position - 1] == recipe_type:
                new_recipes_list = set(new_recipes_list)
                new_recipes_list.add(neigh_position)
            
            if morid_nodes_list[neigh_position - 1] == morid_type:
                if morids_recipes[neigh_position].issubset(new_recipes_list):            
                    new_morids_list = set(new_morids_list)
                    new_morids_list.add(neigh_position)
            
            if hard_nodes_list[neigh_position - 1] == hard_type:
                wait_time = new_hard_nodes_list[neigh_position]
                new_hard_nodes_list[neigh_position] += 1
                
            new_state = State(neigh_position, state.spent_time + 1, wait_time, new_path, new_hard_nodes_list, new_recipes_list, new_morids_list)
            
            if(is_goal(new_state)):
                utils.print_state(new_state, file_name, state_counter)
                return
            
            if new_state not in explored_states and new_state not in frontier_states:
                frontier_states.appendleft(new_state)
                state_counter += 1   


## Test BFS

In [7]:
bfs('Test 1.txt')
bfs('Test 2.txt')
bfs('Test 3.txt')
bfs('Test 4.txt')
bfs('Test 5.txt')


testing bfs for Test 1 started ...
Path : 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
States Count : 33
Spent Time : 8
executed in 0.0 sec
testing ends

testing bfs for Test 2 started ...
Path : 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
States Count : 88
Spent Time : 8
executed in 0.0 sec
testing ends

testing bfs for Test 3 started ...
Path : 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
States Count : 844
Spent Time : 13
executed in 0.08403420448303223 sec
testing ends

testing bfs for Test 4 started ...
Path : 28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 23 -> 28 -> 23 -> 5 -> 7 -> 29
States Count : 3470
Spent Time : 12
executed in 3.4733948707580566 sec
testing ends

testing bfs for Test 5 started ...
Path : 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
States Count : 3439
Spent Time : 21
executed in 1.7059333324432373 sec
testing ends



## Test BFS Results
| --- | Min Spent Time | Satates Seen | Avg Exec Time |
| --- | --- | --- | --- |
| Test 1 | 8 | 33 | 0.00099s |
| Test 2 | 8 | 88 | 0.00100s |
| Test 3 | 13 | 844 |  0.08438s |
| Test 4 | 12 | 3470 |  3.37352s |
| Test 5 | 21 | 3439 | 1.67051s |

## IDS

## Modeling IDS
We use state for path finding and state hase many fields (shown in its dataclass)
Also we use frontier_states (stack) and explored_states (list)

- Initial State : State(position : init_position ,
                        spent_time : 0 ,
                        wait_time : 0 ,
                        path : empty_list ,
                        hard_nodes : empty_list_dict ,
                        done_recipes : empty_set ,
                        done_morids : empty_set)
                        
- Actions : we can append neighs of last node of frontier_states to first of that
- Transition Model : we expand nodes that are not expanded and not in frontier_states to achive goal state
- Goal state : in goal state we have done_morids is morid_nodes_num and wait_time is 0
- Path Cost : spent_time must be path cost

In [8]:
def is_goal(state):
    return True if len(state.done_morids) == morid_nodes_num and state.wait_time == 0 else False

@utils.timer_func
def ids(file_name):
    state_counter = 1
    file_name_ = file_name.replace('.txt', '')
    print(f"testing {bcolors.BLUE}ids{bcolors.WHITE} for {bcolors.GREEN}{file_name_}{bcolors.WHITE} started ...")
    read_file(file_name)
    frontier_states = []
    depth = 1
    while(True):
        if len(frontier_states) == 0:
            frontier_states = []
            first_state = State(seyed_position, 0, 0, [seyed_position], hard_nodes, [], [])
            frontier_states.append(first_state)
            depth += 1
            continue
        
        state = frontier_states.pop()
        
        if state.spent_time > depth:
            continue

        ### decrease wait time
        if state.wait_time != 0:
            state.wait_time -= 1
            state.spent_time += 1
            frontier_states.append(state)
            continue
        
        neighs_position = neighs[state.position - 1]
        neighs_position.reverse()
        for neigh_position in neighs_position:
            
            new_path = state.path.copy()
            new_morids_list = state.done_morids
            new_recipes_list = state.done_recipes
            new_hard_nodes_list = state.hard_nodes.copy()
            wait_time = 0
            
            new_path.append(neigh_position)
            
            if recipe_nodes_list[neigh_position - 1] == recipe_type:
                new_recipes_list = set(new_recipes_list)
                new_recipes_list.add(neigh_position)
                
            if morid_nodes_list[neigh_position - 1] == morid_type:
                if morids_recipes[neigh_position].issubset(new_recipes_list):            
                    new_morids_list = set(new_morids_list)
                    new_morids_list.add(neigh_position)
            
            if hard_nodes_list[neigh_position - 1] == hard_type:
                wait_time = new_hard_nodes_list[neigh_position]
                new_hard_nodes_list[neigh_position] += 1
                
            new_state = State(neigh_position, state.spent_time + 1, wait_time, new_path, new_hard_nodes_list, new_recipes_list, new_morids_list)
            
            if(is_goal(new_state)):
                utils.print_state(new_state, file_name, state_counter)
                return
            
            if new_state not in frontier_states:
                frontier_states.append(new_state) 
                state_counter += 1  


## Test IDS

In [9]:
ids('Test 1.txt')
ids('Test 2.txt')
ids('Test 3.txt')

testing ids for Test 1 started ...
Path : 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
States Count : 914
Spent Time : 8
executed in 0.0 sec
testing ends

testing ids for Test 2 started ...
Path : 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
States Count : 3793
Spent Time : 8
executed in 0.013000249862670898 sec
testing ends

testing ids for Test 3 started ...
Path : 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
States Count : 68476
Spent Time : 13
executed in 0.17401337623596191 sec
testing ends



## Test IDS Results
| --- | Min Spent Time | Satates Seen | Avg Exec Time |
| --- | --- | --- | --- |
| Test 1 | 8 | 914 | 0.00285s |
| Test 2 | 8 | 3793 | 0.01099s |
| Test 3 | 13 | 68476 | 0.16408s |

## A*

heuritic function for nodes is : number of morid and recipe nodes that are not statisfied

and for be consistant , we use spent_time and heuristic function that means we use f(n) = g(n) + h(n) for pushing in min-heap and use A* algorithm. it is consistant because for all Ai that are parent of node Aj (both are in path like path = [A1 , A2 , ... , Ai , ... , Aj , ... , An]) we have :

$h(A_{i} - A{j}) \leq cost(A_{i} to A_{j})$

that cause of we use f(n) = g(n) + h(n).


## Modeling A*
We use state for path finding and state hase many fields (shown in its dataclass)
Also we use frontier_states (min-heap) and explored_states (list)

- Initial State : State(position : init_position ,
                        spent_time : 0 ,
                        wait_time : 0 ,
                        path : empty_list ,
                        hard_nodes : empty_list_dict ,
                        done_recipes : empty_set ,
                        done_morids : empty_set)
                        
- Actions : we expand poped node from frontier_states that orders first by its priority and then by its entering order in min-heap
- Transition Model : we expand nodes that are not expanded and not in frontier_states to achive goal state
- Goal state : in goal state we have done_morids is morid_nodes_num and wait_time is 0
- Path Cost : spent_time must be path cost

In [10]:
def is_goal(state):
    return True if len(state.done_morids) == morid_nodes_num and state.wait_time == 0 else False

def get_heuristic(state):
    return recipe_nodes_num - len(state.done_recipes) + morid_nodes_num - len(state.done_morids) + state.wait_time    

@utils.timer_func
def a_star(file_name):
    state_counter = 1
    file_name_ = file_name.replace('.txt', '')
    print(f"testing {bcolors.BLUE}a_star{bcolors.WHITE} for {bcolors.GREEN}{file_name_}{bcolors.WHITE} started ...")
    read_file(file_name)
    frontier_states = []
    heapq.heapify(frontier_states)
    explored_states = []
    first_state = State(seyed_position, 0, 0, [seyed_position], hard_nodes, [], [])
    heapq.heappush(frontier_states, (get_heuristic(first_state) + 0, 0, first_state))
    while(True):
        state = heapq.heappop(frontier_states)[2]
        if(is_goal(state)):
            utils.print_state(state, file_name, state_counter)
            return
        
        ### decrease wait time
        if state.wait_time != 0:
            state.wait_time -= 1
            state.spent_time += 1
            heapq.heappush(frontier_states, (get_heuristic(state) + state.spent_time, state_counter, state))
            state_counter += 1
            continue
        
        explored_states.append(state)
        neighs_position = neighs[state.position - 1]
        for neigh_position in neighs_position:
            
            new_path = state.path.copy()
            new_morids_list = state.done_morids
            new_recipes_list = state.done_recipes
            new_hard_nodes_list = state.hard_nodes.copy()
            wait_time = 0
            
            new_path.append(neigh_position)
            
            if recipe_nodes_list[neigh_position - 1] == recipe_type:
                new_recipes_list = set(new_recipes_list)
                new_recipes_list.add(neigh_position)
            
            if morid_nodes_list[neigh_position - 1] == morid_type:
                if morids_recipes[neigh_position].issubset(new_recipes_list):            
                    new_morids_list = set(new_morids_list)
                    new_morids_list.add(neigh_position)
            
            if hard_nodes_list[neigh_position - 1] == hard_type:
                wait_time = new_hard_nodes_list[neigh_position]
                new_hard_nodes_list[neigh_position] += 1
                
            new_state = State(neigh_position, state.spent_time + 1, wait_time, new_path, new_hard_nodes_list, new_recipes_list, new_morids_list)
            
            if(is_goal(new_state)):
                utils.print_state(new_state, file_name, state_counter)
                return
            
            if new_state not in explored_states:
                is_in_frontier = False
                for frontier_state in frontier_states:
                    if new_state == frontier_state[2]:
                        is_in_frontier = True
                        break
                if not is_in_frontier:
                    heapq.heappush(frontier_states, (get_heuristic(new_state) + new_state.spent_time, state_counter, new_state)) 
                    state_counter += 1


## Test A*

In [11]:
a_star('Test 1.txt')
a_star('Test 2.txt')
a_star('Test 3.txt')
a_star('Test 4.txt')
a_star('Test 5.txt')

testing a_star for Test 1 started ...
Path : 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
States Count : 34
Spent Time : 8
executed in 0.0 sec
testing ends

testing a_star for Test 2 started ...
Path : 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
States Count : 53
Spent Time : 8
executed in 0.0 sec
testing ends

testing a_star for Test 3 started ...
Path : 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
States Count : 459
Spent Time : 13
executed in 0.013000011444091797 sec
testing ends

testing a_star for Test 4 started ...
Path : 28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 23 -> 5 -> 7 -> 29 -> 22 -> 28
States Count : 1663
Spent Time : 12
executed in 0.4660983085632324 sec
testing ends

testing a_star for Test 5 started ...
Path : 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
States Count : 3085
Spent Time : 21
executed in 1.3016459941864014 sec
testing ends



## Test A* Results
| --- | Min Spent Time | Satates Seen | Avg Exec Time |
| --- | --- | --- | --- |
| Test 1 | 8 | 34 | 0.00099s |
| Test 2 | 8 | 53 | 0.00102s |
| Test 3 | 13 | 459 | 0.02064s |
| Test 4 | 12 | 1663 | 0.49756s |
| Test 5 | 21 | 3085 | 1.34748s |

## Weighted A*

we use $alpha$ that multipy in h(n).

## Modeling Weighted A*

It is like A* but we use $alpha$ to increasing priority of nodes


In [12]:

def is_goal(state):
    return True if len(state.done_morids) == morid_nodes_num and state.wait_time == 0 else False

def get_heuristic(state):
    return recipe_nodes_num - len(state.done_recipes) + morid_nodes_num - len(state.done_morids) + state.wait_time    

@utils.timer_func
def weighted_a_star(alpha, file_name):
    file_name_ = file_name.replace('.txt', '')
    print(f"testing {bcolors.BLUE}weighted_a_star{bcolors.WHITE} for {bcolors.GREEN}{file_name_}{bcolors.WHITE} started ...")
    read_file(file_name)
    frontier_states = []
    heapq.heapify(frontier_states)
    explored_states = []
    first_state = State(seyed_position, 0, 0, [seyed_position], hard_nodes, [], [])
    heapq.heappush(frontier_states, (get_heuristic(first_state) * alpha + 0, 0, first_state))
    state_counter = 1
    while(True):
        state = heapq.heappop(frontier_states)[2]
        if(is_goal(state)):
            utils.print_state(state, file_name, state_counter)
            return
        
        ### decrease wait time
        if state.wait_time != 0:
            state.wait_time -= 1
            state.spent_time += 1
            heapq.heappush(frontier_states, (get_heuristic(state) * alpha + state.spent_time, state_counter, state))
            state_counter += 1
            continue
        
        explored_states.append(state)
        neighs_position = neighs[state.position - 1]
        for neigh_position in neighs_position:
            
            new_path = state.path.copy()
            new_morids_list = state.done_morids
            new_recipes_list = state.done_recipes
            new_hard_nodes_list = state.hard_nodes.copy()
            wait_time = 0
            
            new_path.append(neigh_position)
            
            if recipe_nodes_list[neigh_position - 1] == recipe_type:
                new_recipes_list = set(new_recipes_list)
                new_recipes_list.add(neigh_position)
            
            if morid_nodes_list[neigh_position - 1] == morid_type:
                if morids_recipes[neigh_position].issubset(new_recipes_list):            
                    new_morids_list = set(new_morids_list)
                    new_morids_list.add(neigh_position)
            
            if hard_nodes_list[neigh_position - 1] == hard_type:
                wait_time = new_hard_nodes_list[neigh_position]
                new_hard_nodes_list[neigh_position] += 1
                
            new_state = State(neigh_position, state.spent_time + 1, wait_time, new_path, new_hard_nodes_list, new_recipes_list, new_morids_list)
            
            if(is_goal(new_state)):
                utils.print_state(new_state, file_name, state_counter)
                return
            
            if new_state not in explored_states:
                is_in_frontier = False
                for frontier_state in frontier_states:
                    if new_state == frontier_state[2]:
                        is_in_frontier = True
                        break
                if not is_in_frontier:
                    heapq.heappush(frontier_states, (get_heuristic(new_state) * alpha + new_state.spent_time, state_counter, new_state)) 
                    state_counter += 1
        

## Weighted A* 1

In [13]:
weighted_a_star(2, 'Test 1.txt')
weighted_a_star(2, 'Test 2.txt')
weighted_a_star(2, 'Test 3.txt')
weighted_a_star(2, 'Test 4.txt')
weighted_a_star(2, 'Test 5.txt')

testing weighted_a_star for Test 1 started ...
Path : 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
States Count : 28
Spent Time : 8
executed in 0.0 sec
testing ends

testing weighted_a_star for Test 2 started ...
Path : 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
States Count : 51
Spent Time : 8
executed in 0.0 sec
testing ends

testing weighted_a_star for Test 3 started ...
Path : 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
States Count : 48
Spent Time : 13
executed in 0.0 sec
testing ends

testing weighted_a_star for Test 4 started ...
Path : 28 -> 19 -> 3 -> 11 -> 24 -> 9 -> 23 -> 5 -> 7 -> 29 -> 20 -> 13 -> 19 -> 28
States Count : 102
Spent Time : 13
executed in 0.0 sec
testing ends

testing weighted_a_star for Test 5 started ...
Path : 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
States Count : 1758
Spent Time : 21
executed in 0.2862064838409424 sec
testing ends



## Weighted A* 1 Results
| --- | Min Spent Time | Satates Seen | Avg Exec Time |
| --- | --- | --- | --- |
| Test 1 | 8 | 28 | 0.00400s |
| Test 2 | 8 | 51 | 0.00760s |
| Test 3 | 13 | 48 | 0.00699s |
| Test 4 | 13 | 102 | 0.00930s |
| Test 5 | 21 | 1758 | 0.54576s |

## Weighted A* 2

In [14]:
weighted_a_star(3, 'Test 1.txt')
weighted_a_star(3, 'Test 2.txt')
weighted_a_star(3, 'Test 3.txt')
weighted_a_star(3, 'Test 4.txt')
weighted_a_star(3, 'Test 5.txt')

testing weighted_a_star for Test 1 started ...
Path : 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
States Count : 24
Spent Time : 8
executed in 0.0 sec
testing ends

testing weighted_a_star for Test 2 started ...
Path : 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8 -> 10 -> 8 -> 5 -> 7
States Count : 45
Spent Time : 12
executed in 0.0 sec
testing ends

testing weighted_a_star for Test 3 started ...
Path : 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 10 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11
States Count : 39
Spent Time : 14
executed in 0.0 sec
testing ends

testing weighted_a_star for Test 4 started ...
Path : 28 -> 19 -> 3 -> 11 -> 24 -> 9 -> 23 -> 5 -> 7 -> 29 -> 20 -> 13 -> 19 -> 28
States Count : 96
Spent Time : 13
executed in 0.0 sec
testing ends

testing weighted_a_star for Test 5 started ...
Path : 40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 9 -> 49 -> 47 -> 49 -> 12 -> 16 -> 12 -> 50 -> 25
States Count : 836
Spent Time : 23
executed in 0.07004594802856445 sec
t

## Weighted A* 2 Results
| --- | Min Spent Time | Satates Seen | Avg Exec Time |
| --- | --- | --- | --- |
| Test 1 | 8 | 24 |  0.00493s |
| Test 2 | 12 | 45 | 0.00099s |
| Test 3 | 14 | 39 | 0.00100s |
| Test 4 | 13 | 96 | 0.00099s |
| Test 5 | 23 | 836 | 0.06732s |